In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pwd

Mounted at /content/drive
/content


In [ ]:
import os
os.chdir(r'/content/drive/Othercomputers/我的笔记本电脑/文本分析大作业/')
os.getcwd()

'/content/drive/Othercomputers/我的笔记本电脑/文本分析大作业'

找一个自己喜欢的网络文档，进行分词后，开展如下练习
（3）构建Word2Vec模型，将代码贴于下方：


1. 分别基于上述三种模型，计算词语相似度。
找2个词语，利用三种算法，计算其最相似的前5个词语。
请比较三种算法的异同。并说明这三种算法可能应用于什么场景。


In [ ]:
import jieba 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from PIL import Image
import re 

In [ ]:
def load_data():
    """读取data,返回一个已经被jieba分好的字符串"""
    if os.path.exists("data/data.txt"):
        with open("data/data.txt","r",encoding="utf-8") as txt:
            b=txt.read()
        return b
    else:
        with open("data\Chinese-data.txt","r",encoding="utf-8") as txt:
            a=txt.read()
            a=a.replace("\n","").replace("\r","")
        b=" ".join(jieba.cut(a))
        with open("data/data.txt","w",encoding="utf-8") as f:
            f.write(b)
        return b

b=load_data()
b[:100]

'版权 信息 COPYRIGHT 书名 ： 精灵 宝 钻 作者 ： 【 英 】 托尔金 （ Tolkien ,   J . R . R . ） 出版社 ： 上海人民出版社 · 世纪 文景 出版 时间 ：'

In [ ]:
## 去掉停用词，包括data/停用词.txt中的以及长度为1的词(其实就是标点符号和“的”等非常没用的词)
stoplist = list(pd.read_csv('data/停用词.txt',names=['w'],sep='\t',encoding='utf-8',engine='python').w)
blist=b.split(" ")
txt_no_stoplist=[i for i in blist if i not in stoplist and len(i)>1]
df=pd.value_counts(txt_no_stoplist)
str_no_stoplist=" ".join(txt_no_stoplist)
## 词频统计结果
df

弗罗多         2109
精灵          1794
一个          1645
甘道夫         1423
山姆          1397
            ... 
讨老婆            1
搞不懂            1
斯不假            1
根茎             1
RESERVED       1
Length: 41576, dtype: int64

In [ ]:
## 用正则表达式解决章节拆分之问题，我是神经病，竟然用这种反人类的语言。
temp=re.split(r"(第\w+章)",b)
b_split=[]
for i,itemp in enumerate(temp):
    if re.match(r"(第\w+章)",itemp):
        b_split.append(itemp+temp[i+1])

In [21]:
from gensim.models.word2vec import Word2Vec

n_dim = 300 # 指定向量维度，大样本量时300~500较好

w2vmodel = Word2Vec(size = n_dim, min_count = 10) # size改为vector_size
w2vmodel.build_vocab([blist]) # 生成词表
w2vmodel

In [22]:
%time w2vmodel.train(b_split, \
               total_examples = w2vmodel.corpus_count, epochs = 100)

CPU times: user 5min 47s, sys: 1.79 s, total: 5min 49s
Wall time: 2min 2s


(79156679, 229121600)

In [24]:
# 查看向量大小，内容
print(w2vmodel.wv["费艾诺"].shape)
w2vmodel.wv["费艾诺"]

(300,)


array([ 1.57637871e-03,  1.64053636e-04,  3.46456742e-04, -6.33350923e-04,
       -1.49544608e-03, -1.65338430e-03,  8.59894906e-04, -1.34163944e-03,
       -1.11019809e-03,  1.39150512e-03,  8.86595342e-04,  8.75828206e-04,
        1.36892404e-03,  8.39200802e-04, -3.87810520e-04, -1.15117698e-03,
        6.43581909e-04,  2.97205173e-04, -1.20143837e-03, -2.50914629e-04,
       -9.49112349e-04, -8.45785835e-04, -1.13627560e-04,  1.33504870e-03,
       -1.51217356e-03, -3.12404911e-04, -1.61124952e-03,  9.78575088e-04,
       -6.29911898e-04,  1.11080194e-03,  1.31712423e-03, -1.25401979e-03,
        1.36663578e-03,  1.80029907e-04,  3.47901427e-04,  1.36724906e-03,
        1.05882820e-03,  1.41519401e-03, -9.85590974e-04,  1.37852191e-03,
        1.04004540e-03, -7.41042095e-05,  1.51813461e-03, -9.40014550e-04,
        7.01289333e-04,  1.48845173e-03,  4.09595843e-04,  1.12975005e-03,
        1.03638985e-03,  4.11092653e-04,  3.17816273e-04, -1.43872073e-03,
        1.26954354e-03,  

In [25]:
print("费艾诺最相似的词：", w2vmodel.wv.most_similar("费艾诺", topn=5))
print("甘道夫最相似的词：", w2vmodel.wv.most_similar("甘道夫", topn=5))
print("魔戒最相似的词：", w2vmodel.wv.most_similar("魔戒", topn=5))
# print("洪七公最相似的词：", w2vmodel.wv.most_similar("洪七公", topn=5))
# print("黄药师最相似的词：", w2vmodel.wv.most_similar("黄药师", topn=5))

费艾诺最相似的词： [('唱道', 0.1998140662908554), ('山毛榉', 0.1852053552865982), ('武装', 0.1829865276813507), ('欧西', 0.18067190051078796), ('不想', 0.17966967821121216)]
甘道夫最相似的词： [('羽箭', 0.2179679423570633), ('一间', 0.19735974073410034), ('打碎', 0.19246597588062286), ('沉没', 0.19115421175956726), ('呼姆', 0.19024480879306793)]
魔戒最相似的词： [('这才', 0.20685261487960815), ('抱', 0.19538399577140808), ('格', 0.17848056554794312), ('地面', 0.17765960097312927), ('有过', 0.17700985074043274)]


## 深度学习模型

In [ ]:
!python3 main.py

自己组合数据...
使用jieba分词...
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.780 seconds.
Prefix dict has been built successfully.
jieba总共分出来了 4315 个词
分词完成,最终入选的字和词有 13028 个...
获得了 (8384, 125) 维度的漂亮数据...
自己组合数据完成！祝您生活愉快...
user config:
jieba_rate 0.01
data_path data/Chinese-data.txt
pickle_path data/all_data.npz
author None
constrain None
category Marx
lr 0.001
weight_decay 0.0001
use_gpu True
iter_consecutive True
max_epoch 50
lr_decay 1
lr_decay_step 20
clipping_grad True
clipping_theta 0.01
batch_size 64
maxlen 125
print_freq 40
save_freq 20
max_gen_len 200
model_path None
prefix_words 图奥出身的那支民族在森林里、高地上游荡，他们不知道也不歌颂大海。不过，图奥不和他们一起生活，而是独自住在一座名叫米斯林的湖附近，有时在湖畔的林中打猎，有时在岸边用他以熊筋和木头做成的简陋竖琴弹奏乐曲。众人听说他那粗犷的歌声含有力量，纷纷从远近前来听他弹唱，但是图奥不再歌唱，动身去了偏僻荒凉的地方。他在那里见识了诸多新奇的事物，并且从流浪的诺多族那里学到了他们的语言与传承学识，但他命中注定不会永远留在那片森林中。
start_words 刚多林的陷落
padding pre
truncating post
acrostic True
criterion CrossEntropyLoss()
device cuda
training on  cuda
Train Epoch: 0